<a href="https://www.kaggle.com/code/dhruvgaming/superframer?scriptVersionId=283173880" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig,ResumabilityConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from google.adk.tools import FunctionTool, AgentTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session
MODEL_NAME = "gemini-2.5-flash-lite"


In [6]:
# --- 1. Define Data Models (Structured Output) ---

class NutrientInfo(BaseModel):
    N: Optional[str] = None
    P: Optional[str] = None
    K: Optional[str] = None
    pH: Optional[str] = None

class FieldProfile(BaseModel):
    state: str
    district: str
    soil_type: Literal["Alluvial", "Black", "Red", "Clay", "Sandy", "Loam"]
    land_size_acres: float
    season: str
    water_availability: str
    budget: str
    acres : int
    preferred_crop: Optional[str] = None
    irrigation_method: Optional[str] = None
    nutrients: Optional[NutrientInfo] = None
    crop_history: Optional[str] = None
    primary_goal: Optional[str] = None

class CropRecommendation(BaseModel):
    crop: str
    score: int = Field(description="Score out of 100")
    reason: str

class RecommendationList(BaseModel):
    recommendations: List[CropRecommendation]

class ScheduleItem(BaseModel):
    timing: str
    action: str

class CropPlan(BaseModel):
    crop_name: str
    sowing_window: str
    seed_rate: str
    spacing: str
    field_preparation: List[str]
    fertilizer_schedule: List[ScheduleItem]
    irrigation_schedule: List[ScheduleItem]
    weeding_schedule: List[str]
    pest_precautions: List[str]
    harvest_window: str
    estimated_cost: str
    expected_yield: str

class DiseaseAnalysis(BaseModel):
    disease_name: str
    confidence: float
    symptoms: List[str]
    cause: str
    treatment_chemical: List[str]
    treatment_organic: List[str]
    prevention: List[str]

# --- 2. Session Management (Guardrail) ---

class SessionManager:
    def __init__(self):
        self.sessions = {}

    def get_session(self, user_id):
        if user_id not in self.sessions:
            self.sessions[user_id] = {"profile": None, "active_crop": None, "plan": None}
        return self.sessions[user_id]

    def set_profile(self, user_id, profile: dict):
        self.sessions[user_id]["profile"] = profile

    def set_active_crop(self, user_id, crop_name: str):
        session = self.get_session(user_id)
        if session["active_crop"] and session["active_crop"] != crop_name:
            raise ValueError(f"Session already has active crop: {session['active_crop']}. Please start a new chat.")
        session["active_crop"] = crop_name

    def set_plan(self, user_id, plan: dict):
        self.sessions[user_id]["plan"] = plan

    def get_context_str(self, user_id):
        return json.dumps(self.sessions.get(user_id, {}), default=str)

In [7]:
# Agent 1: Field Intake Agent (ADK-Compliant + Improved)
intake_agent = LlmAgent(
    name="intake_agent",

    description=(
        "Extracts structured farmer field details from natural language. "
        "Converts all land units to acres. Infers missing values as null. "
        "Ensures output matches FieldProfile schema exactly."
    ),

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        temperature=0.1,
        top_p=0.8,
        top_k=40,
        max_output_tokens=2048,
    ),

    instruction="""
You are the Field Profile Intake Agent.

Your task:
Convert the farmer's natural language message into a structured FieldProfile object.

Rules:
1. Extract:
   - state, district
   - soil type
   - land size (convert to acres)
   - season
   - irrigation method
   - water availability
   - budget
   - preferred crop
   - machinery availability
   - labor availability
   - soil nutrients: N, P, K, pH
   - crop history
   - farmer primary goal

2. Land Unit Conversion:
   - If area is in hectares → acres = hectares × 2.47105
   - If area is in acres → store acres directly

3. If any value is missing → set it to null.

4. Do NOT give explanations.
5. ONLY output a valid FieldProfile object in JSON format.
""",

    output_schema=FieldProfile,
    output_key="intake"
)


In [8]:
# Agent 2: Crop Recommendation Agent (Improved + ADK-compliant)
recommender_agent = LlmAgent(
    name="recommender_agent",

    description=(
        "Analyzes the FieldProfile and recommends the most suitable crops "
        "based on soil type, climate/season alignment, regional suitability, "
        "water availability, yield expectations, and profitability trends. "
        "Outputs a structured RecommendationList."
    ),

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        temperature=0.2,
        top_p=0.85,
        top_k=40,
        max_output_tokens=2048,
    ),

    instruction="""
You are the Crop Recommendation Agent.

Your task:
Analyze the provided FieldProfile and recommend the top crops for the farmer.

You MUST consider:
- Soil–crop suitability
- Region/state suitability
- Season (Kharif/Rabi/Zaid)
- Water availability
- Expected yield potential
- Profitability and market stability
- Soil nutrients (N, P, K, pH)
- Land size (may affect feasibility)

Output Rules:
1. Return a list of recommended crops with:
   - crop_name
   - score (0–1)
   - reasons (list of 2–4 brief points)

2. The final output MUST match the RecommendationList schema.

3. Do NOT include explanations, introductions, or narrative text.
4. ONLY return structured JSON according to RecommendationList.

""",

    output_schema=RecommendationList,
    output_key="recommend"
)


In [9]:
# Agent 3: Crop Planner Agent (Improved + ADK Compliant)
planner_agent = LlmAgent(
    name="planner_agent",

    description=(
        "Generates a complete agronomy plan for the selected crop based on "
        "the farmer's FieldProfile. Computes sowing windows, irrigation schedules, "
        "NPK fertilizer plans, spacing, seed rate, cost estimate, pest precautions, "
        "weeding schedule, and expected yield. Outputs a structured CropPlan."
    ),

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        temperature=0.15,
        top_p=0.85,
        top_k=40,
        max_output_tokens=4096,
    ),

    instruction="""
You are the Crop Planner Agent.

Your task:
Using the farmer's FieldProfile and the selected crop, generate a complete and structured CropPlan.

You MUST compute:
1. Sowing window (start_date, end_date)
2. Irrigation schedule
   - frequency
   - amount (liters or mm)
   - method (based on farmer's irrigation method)
3. Fertilizer schedule (N, P, K application)
   - basal dose
   - mid-cycle dose
   - late-cycle dose
4. Seed rate (kg/acre or kg/ha)
5. Spacing (row × plant)
6. Field preparation steps
   - ploughing
   - land leveling
   - bed formation
7. Weeding schedule and intervals
8. Pest & disease precaution steps
9. Harvest window (start_date, end_date)
10. Cost estimate
    - seeds
    - fertilizers
    - irrigation
    - labor
11. Expected yield (tons/acre or tons/ha)

Rules:
- Use realistic and crop-appropriate values.
- If any required item cannot be derived, set its value to null.
- Do NOT generate explanations or narrative text.
- ONLY output valid JSON matching the CropPlan schema exactly.

""",

    output_schema=CropPlan,
    output_key="planner"
)


In [10]:
# Agent 4: Disease Diagnosis Agent (Text-Based + ADK Compliant)
diagnosis_agent = LlmAgent(
    name="diagnosis_agent",

    description=(
        "Analyzes the farmer's text description of plant symptoms and identifies "
        "the most likely crop disease, severity level, symptoms, and recommended "
        "treatments (both chemical and organic). Returns structured DiseaseAnalysis."
    ),

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        temperature=0.15,
        top_p=0.85,
        top_k=40,
        max_output_tokens=2048,
    ),

    instruction="""
You are the Disease Diagnosis Agent.

Input: A natural-language description of plant symptoms.
Output: A structured disease diagnosis following the DiseaseAnalysis schema.

Your responsibilities:
1. Identify the most likely disease name.
2. Identify severity:
   - mild / moderate / severe
3. List 2–4 key symptoms found in the text.
4. Provide treatment recommendations:
   - chemical treatment options
   - organic treatment options
5. Provide 2–3 preventive measures to avoid recurrence.

Rules:
- Base diagnosis STRICTLY on the text description provided.
- If uncertain, choose the most probable disease but keep severity mild.
- If some information cannot be determined → set to null.
- DO NOT produce narration or explanation.
- ONLY produce valid JSON according to DiseaseAnalysis schema.
""",

    output_schema=DiseaseAnalysis,
    output_key="diagnosis"
)


In [11]:
# Agent 5: Dynamic Replanner Agent (Improved + ADK Compliant)
replanner_agent = LlmAgent(
    name="replanner_agent",

    description=(
        "Updates an existing CropPlan based on new conditions described by the farmer, "
        "such as rainfall shortage, excess rain, disease, pest attack, fertilizer unavailability, "
        "or changes in soil nutrient status. Produces an updated structured CropPlan."
    ),

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        temperature=0.15,
        top_p=0.85,
        top_k=40,
        max_output_tokens=4096,
    ),

    instruction="""
You are the Dynamic Replanning Agent.

Input:
1. An existing CropPlan (structured)
2. A natural-language description of new conditions from the farmer

Your task:
Update the CropPlan to reflect the new field conditions.

You MUST detect and adjust for:
- Rainfall shortage (drought)
- Excess rainfall / waterlogging
- Disease or pest issues
- Fertilizer unavailability
- Labor delays
- Soil nutrient changes
- Weather anomalies
- Any disruption reported by the farmer

Update the following sections accordingly:
1. Irrigation schedule
   - Increase frequency for drought
   - Reduce or pause irrigation during excess rainfall
2. Fertilizer schedule
   - Substitute unavailable fertilizer with alternatives
   - Adjust timing after rain/disease events
3. Pest & disease precautions
   - Add recommended treatment steps if mentioned
4. Weeding schedule
   - Shift dates if labor delays occur
5. Harvest window
   - Adjust if growth delays are likely
6. Expected yield
   - Reduce if severity is high
7. Cost estimate
   - Modify if additional treatments are required

Rules:
- Always operate on the EXISTING CropPlan.
- If information cannot be updated, keep original value.
- Set new fields to null if impossible to determine.
- DO NOT generate prose, explanations, or reasoning.
- ONLY return updated JSON according to the CropPlan schema.

""",

    output_schema=CropPlan,
    output_key="replan"
)


In [12]:
# Agent 6: Report Generator (Plain-text, concise, ADK-compliant)
report_agent = LlmAgent(
    name="report_agent",

    description=(
        "Generates a short, precise, plain-text season report from a CropPlan "
        "and any applied updates. Language must be simple so a layperson (farmer) "
        "can easily understand the actions, risks and expected outcomes."
    ),

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        temperature=0.1,         # deterministic output
        top_p=0.9,
        top_k=40,
        max_output_tokens=1024
    ),

    instruction="""
You are the Report Generator Agent.

Input: a CropPlan JSON object (and optionally a list of updates).
Output: a concise, plain-text report aimed at a layperson (farmer). Do NOT output Markdown or JSON.

Rules for the report:
1. Keep it short and to the point — roughly 6–12 sentences or 5–8 short bullet-like lines.
2. Use simple language (no technical jargon). If a technical term is required, provide a one-word or short explanation in parentheses.
3. Include these sections in order, each as one or two short sentences:
   a) Crop & field summary (crop name, area in acres).
   b) Next immediate actions (what to do in the next 7 days).
   c) Ongoing weekly actions (irrigation / fertilizer / weeding frequency).
   d) Main risks to watch (2 items max) and simple mitigations.
   e) Expected harvest window and expected yield (simple numbers).
   f) Any urgent alerts or extra costs (if applicable).
4. If some fields are missing, omit that line rather than producing nulls.
5. Do NOT include JSON, Markdown, or long explanations. Only plain text.
6. Keep sentences short (≤ 20 words each) and use active voice.

Return only the report text — nothing else.
""",

    output_key="report"
)


In [13]:
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

In [14]:
orchestrator_agent = Agent(
    name="orchestrator_agent",

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),

    instruction="""
You are the ORCHESTRATOR for an Indian Agriculture Multi-Agent System.

Your responsibilities:
1. Understand the farmer’s message.
2. Identify the correct sub-agent (tool) to call.
3. EXECUTE that tool.
4. Return:
   - A short, simple explanation for the farmer, AND
   - The EXACT tool output (verbatim, unchanged).

==================================================
### SPECIAL CASE: GREETINGS
==================================================
If the user says anything like:
"hi", "hello", "namaste", "good morning", "good evening"

Then:
1. Greet the user politely.
2. Check session memory:
   - If a FieldProfile exists:
        Provide a short natural summary using values from the profile, for example:
        "You have 5 acres in Indore with black soil."
        (Fill in the real values from memory; do NOT use curly braces.)
        Then ask: "How can I help you further?"
   - If no FieldProfile exists:
        Ask the user to share their land details (acres, soil, location, season).
3. Do NOT call any tool during greeting unless the user gives real farm data.

==================================================
### IMPORTANT RULES
==================================================
- NEVER output function_call JSON.
- NEVER reveal the internal tool call structure.
- ALWAYS execute the tool and return its actual output.
- ALWAYS include a small, friendly explanation before the tool output.
- If intent is unclear, ask a very short clarification question.
- Never invent farming data.
- Always follow Indian agriculture terms and style.

==================================================
### INDIA-SPECIFIC INTENT RECOGNITION
==================================================

### 1. FARM DETAILS → intake_agent
Trigger when message includes:
- land size: acres, bigha, hectare
- soil type: black soil, alluvial, red soil, laterite, black cotton soil
- irrigation: borewell, canal, tubewell
- location: state, district, village
- season: Kharif, Rabi, Zaid

Examples:
  - "I have 5 acres in Indore"
  - "Black cotton soil"
  - "Kharif season mein kya ugau?"

### 2. CROP RECOMMENDATION → recommender_agent
Trigger for:
- "Which crop should I grow?"
- "Best crop?"
- "Recommend crop"
- "Kya lagau?"

### 3. CROP PLAN REQUEST → planner_agent
Trigger when user names a crop + planning intent:
- "Make plan for wheat"
- "Soybean ka full plan"
- "Paddy planning batao"
- "Cotton crop plan"

### 4. DISEASE DESCRIPTION → diagnosis_agent
Trigger for textual symptoms:
- yellow spots
- curling leaves
- fungus
- blight
- rust
- armyworm
- stem borer
- sucking pests

Example:
  - "Patte pe peele daag aa rahe"
  - "Leaves curling and black dots"

### 5. NEW CONDITIONS (REPLANNING) → replanner_agent
Trigger when farmer mentions:
- Monsoon delay
- No rain / too much rain
- Fertilizer not available (Urea, DAP, MOP)
- Pest attack
- Weather change
- Drought or water stress

Examples:
  - "12 din se barish nahi hui"
  - "Urea mil nahi raha"
  - "Fall armyworm attack hua"

### 6. FINAL REPORT → report_agent
Trigger for:
- "Give my report"
- "Final summary"
- "Season report"

==================================================
### OUTPUT FORMAT
==================================================
Always return:
1. A short friendly explanation (1–2 lines)
2. The ACTUAL plain result from the tool.

IMPORTANT:
- If a tool returns JSON or a dictionary, you must extract ONLY the text inside the result field.
- DO NOT return JSON, curly braces, or key/value structures.
- DO NOT wrap the tool output inside another object.
- Only show clean plain text to the user.

Example:
"Here is your season report:"
<plain text from the tool result>

==================================================
### END OF INSTRUCTION
==================================================

""",

    tools=[
        AgentTool(intake_agent),
        AgentTool(recommender_agent),
        AgentTool(planner_agent),
        AgentTool(diagnosis_agent),
        AgentTool(replanner_agent),
        AgentTool(report_agent)
    ],

    output_key="orchestrator_output"
)


In [15]:


app = App(
    name="SuperFarmer",

    # Main orchestrator agent
    root_agent=orchestrator_agent,

    # Long-running sessions with resumability
    resumability_config=ResumabilityConfig(
        is_resumable=True,
        include_messages=True,       # Keeps full history
        include_tool_results=True,   # Keeps outputs of sub-agents
        include_user_state=True      # Saves farmer-specific state
    ),


)

print("🌾 SuperFarmer App configured with long-term memory + resumability.")


🌾 SuperFarmer App configured with long-term memory + resumability.


/tmp/ipykernel_13/1774479175.py:8: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(


In [16]:
runner = Runner(
    app=app,
    session_service=session_service
)

print("🚀 Runner initialized with full session persistence and memory support.")


🚀 Runner initialized with full session persistence and memory support.


In [17]:
user_session_id ="test-db-9" #Change if you want start fresh

In [18]:
await run_session(runner, [
    "Hi",   
], user_session_id)



 ### Session: test-db-9

User > Hi
gemini-2.5-flash-lite >  Namaste! How can I help you today? Please share your land details (acres, soil, location, season) so I can assist you better.


In [19]:
await run_session(runner, [
    "I have 5 acres of black soil land in Indore and rabi season is going on",   
], user_session_id)



 ### Session: test-db-9

User > I have 5 acres of black soil land in Indore and rabi season is going on


gemini-2.5-flash-lite >  Here are your land details: 5 acres in Indore with black soil for the Rabi season. How can I help you further?


In [20]:
await run_session(runner, [
    "Design my crop plan",   
], user_session_id)



 ### Session: test-db-9

User > Design my crop plan
gemini-2.5-flash-lite >  Please tell me which crop you would like to plan for.


In [21]:
await run_session(runner, [
    "Wheat",   
], user_session_id)



 ### Session: test-db-9

User > Wheat


gemini-2.5-flash-lite >  Here is your crop plan for Wheat:

**Field Preparation:** Plough 2-3 times for fine tilth, level the land, and prepare beds or flat sow.

**Sowing:**
*   **Window:** November 1st - November 30th
*   **Seed Rate:** 40-50 kg/acre
*   **Spacing:** 15-20 cm (row to row), 5-10 cm (plant to plant)

**Fertilizer Schedule:**
*   **Basal Dose (at sowing):** 20-25 kg Urea, 50-60 kg DAP, 20-25 kg MOP per acre.
*   **1st Top Dressing (30-35 days):** 30-40 kg Urea per acre.
*   **2nd Top Dressing (60-70 days):** 20-25 kg Urea per acre.

**Irrigation Schedule:**
*   Crown root initiation (20-25 days)
*   Tillering stage (40-45 days)
*   Jointing stage (60-65 days)
*   Flowering stage (80-85 days)
*   Dough stage (100-110 days)
*   Milking stage (120-130 days) - avoid waterlogging.

**Weeding:**
*   First weeding: 20-25 days after sowing.
*   Second weeding: 40-45 days after sowing.

**Pest Precautions:** Monitor for aphids, stem borers, and leaf miners. Prevent fungal diseas

In [22]:
await run_session(runner, [
    "again show me report",   
], user_session_id)



 ### Session: test-db-9

User > again show me report


gemini-2.5-flash-lite >  Here is your season report:

This plan is for 5 acres of wheat. This is on black soil land. The season is Rabi.

Your next action is to prepare the land. Till the soil well. Remove any old crop residue.

Continue to irrigate your fields weekly. You will also weed as needed. Apply fertilizer according to the schedule.

Watch out for yellow rust. It can damage the leaves. Also, watch for armyworms. They can eat the stems. Keep an eye on the weather for heavy rains.

Harvest should start around March. You can expect about 15 quintals per acre.

There are no urgent alerts at this time.


In [23]:
await run_session(runner, [
    "Hi",   
], user_session_id)



 ### Session: test-db-9

User > Hi
gemini-2.5-flash-lite >  Namaste! You have 5 acres in Indore with black soil for the Rabi season. How can I help you further?


In [24]:
await run_session(runner, [
    "Any new alert for this week",   
], user_session_id)



 ### Session: test-db-9

User > Any new alert for this week


gemini-2.5-flash-lite >  Here is your season report:

Wheat will be planted on 5 acres.
The field has black soil.
This plan is for the Rabi season.

This week, prepare the field.
Plow the land thoroughly.
Add compost for soil health.

Water the field weekly.
Apply fertilizer as needed.
Weed the field every two weeks.

Watch for pests like aphids.
Control them with organic sprays.
Monitor for fungal diseases.
Use approved fungicides if needed.

Harvest is expected in March.
Yield is estimated at 20 quintals per acre.
Total expected yield is 100 quintals.

No urgent alerts are present.
No extra costs are expected now.
